##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# BERT Question Answer with TensorFlow Lite Model Maker

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/lite/tutorials/model_maker_question_answer"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/tutorials/model_maker_question_answer.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/tutorials/model_maker_question_answer.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/tensorflow/tensorflow/lite/g3doc/tutorials/model_maker_question_answer.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

The [TensorFlow Lite Model Maker library](https://www.tensorflow.org/lite/guide/model_maker) simplifies the process of adapting and converting a TensorFlow model to particular input data when deploying this model for on-device ML applications.

This notebook shows an end-to-end example that utilizes the Model Maker library to illustrate the adaptation and conversion of a commonly-used question answer model for question answer task.

# Introduction to BERT Question Answer Task

The supported task in this library is extractive question answer task, which means given a passage and a question, the answer is the span in the passage. The image below shows an example for question answer.


<p align="center"><img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_squad_showcase.png"  width="500"></p>

<p align="center">
    <em>Answers are spans in the passage (image credit: <a href="https://rajpurkar.github.io/mlx/qa-and-squad/">SQuAD blog</a>) </em>
</p>

As for the model of question answer task, the inputs should be the passage and question pair that are already preprocessed, the outputs should be the start logits and end logits for each token in the passage.
The size of input could be set and adjusted according to the length of passage and question.

## End-to-End Overview


The following code snippet demonstrates how to get the model within a few lines of code. The overall process includes 5 steps: (1) choose a model, (2) load data, (3) retrain the model, (4) evaluate, and (5) export it to TensorFlow Lite format.

```python
# Chooses a model specification that represents the model.
spec = model_spec.get('mobilebert_qa')

# Gets the training data and validation data.
train_data = DataLoader.from_squad(train_data_path, spec, is_training=True)
validation_data = DataLoader.from_squad(validation_data_path, spec, is_training=False)

# Fine-tunes the model.
model = question_answer.create(train_data, model_spec=spec)

# Gets the evaluation result.
metric = model.evaluate(validation_data)

# Exports the model to the TensorFlow Lite format with metadata in the export directory.
model.export(export_dir)
```

The following sections explain the code in more detail.

## Prerequisites

To run this example, install the required packages, including the Model Maker package from the [GitHub repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

In [ ]:
!pip install -q tflite-model-maker

     |████████████████████████████████| 591 kB 5.3 MB/s 
     |████████████████████████████████| 87 kB 6.4 MB/s 
     |████████████████████████████████| 120 kB 45.5 MB/s 
     |████████████████████████████████| 1.2 MB 37.8 MB/s 
     |████████████████████████████████| 636 kB 40.6 MB/s 
     |████████████████████████████████| 211 kB 44.9 MB/s 
     |████████████████████████████████| 64 kB 2.8 MB/s 
     |████████████████████████████████| 840 kB 35.1 MB/s 
     |████████████████████████████████| 1.1 MB 42.4 MB/s 
     |████████████████████████████████| 103 kB 40.2 MB/s 
     |████████████████████████████████| 6.3 MB 40.2 MB/s 
     |████████████████████████████████| 1.1 MB 37.0 MB/s 
     |████████████████████████████████| 99 kB 8.9 MB/s 
     |████████████████████████████████| 37.1 MB 49 kB/s 
     |████████████████████████████████| 352 kB 45.6 MB/s 
     |████████████████████████████████| 200 kB 46.5 MB/s 


Import the required packages.

In [ ]:
import numpy as np
import os

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import question_answer
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.question_answer import DataLoader

The "End-to-End Overview" demonstrates a simple end-to-end example. The following sections walk through the example step by step to show more detail.

## Choose a model_spec that represents a model for question answer

Each `model_spec` object represents a specific model for question answer. The Model Maker currently supports MobileBERT and BERT-Base models.

Supported Model | Name of model_spec | Model Description
--- | --- | ---
[MobileBERT](https://arxiv.org/pdf/2004.02984.pdf)  | 'mobilebert_qa' | 4.3x smaller and 5.5x faster than BERT-Base while achieving competitive results, suitable for on-device scenario.
[MobileBERT-SQuAD](https://arxiv.org/pdf/2004.02984.pdf)  | 'mobilebert_qa_squad' | Same model architecture as MobileBERT model and the initial model is already retrained on [SQuAD1.1](https://rajpurkar.github.io/SQuAD-explorer/).
[BERT-Base](https://arxiv.org/pdf/1810.04805.pdf) | 'bert_qa' | Standard BERT model that widely used in NLP tasks.

In this tutorial, [MobileBERT-SQuAD](https://arxiv.org/pdf/2004.02984.pdf) is used as an example. Since the model is already retrained on [SQuAD1.1](https://rajpurkar.github.io/SQuAD-explorer/), it could coverage faster for question answer task.


In [ ]:
spec = model_spec.get('mobilebert_qa')

## Load Input Data Specific to an On-device ML App and Preprocess the Data

The [TriviaQA](https://nlp.cs.washington.edu/triviaqa/) is a reading comprehension dataset containing over 650K question-answer-evidence triples. In this tutorial, you will use a subset of this dataset to learn how to use the Model Maker library.

To load the data, convert the TriviaQA dataset to the [SQuAD1.1](https://rajpurkar.github.io/SQuAD-explorer/) format by running the [converter Python script](https://github.com/mandarjoshi90/triviaqa#miscellaneous) with `--sample_size=8000` and a set of `web` data. Modify the conversion code a little bit by:
* Skipping the samples that couldn't find any answer in the context document;
* Getting the original answer in the context without uppercase or lowercase.

Download the archived version of the already converted dataset.

In [ ]:
train_data_path = tf.keras.utils.get_file(
    fname='triviaqa-web-train-8000.json',
    origin='https://storage.googleapis.com/download.tensorflow.org/models/tflite/dataset/triviaqa-web-train-8000.json')
validation_data_path = tf.keras.utils.get_file(
    fname='triviaqa-verified-web-dev.json',
    origin='https://storage.googleapis.com/download.tensorflow.org/models/tflite/dataset/triviaqa-verified-web-dev.json')

1179648/1167744 [==============================] - 0s 0us/step


You can also train the MobileBERT model with your own dataset. If you are running this notebook on Colab, upload your data by using the left sidebar.

<img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_question_answer.png" alt="Upload File" width="800" hspace="100">

If you prefer not to upload your data to the cloud, you can also run the library offline by following the [guide](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

Use the `DataLoader.from_squad` method to load and preprocess the [SQuAD format](https://rajpurkar.github.io/SQuAD-explorer/) data according to a specific `model_spec`. You can use either SQuAD2.0 or SQuAD1.1 formats. Setting parameter `version_2_with_negative` as `True` means the formats is SQuAD2.0. Otherwise, the format is SQuAD1.1. By default, `version_2_with_negative` is `False`.

In [ ]:
train_data = DataLoader.from_squad(train_data_path, spec, is_training=True)
validation_data = DataLoader.from_squad(validation_data_path, spec, is_training=False)

## Customize the TensorFlow Model

Create a custom question answer model based on the loaded data. The `create` function comprises the following steps:

1. Creates the model for question answer according to `model_spec`.
2. Train the question answer model. The default epochs and the default batch size are set according to two variables `default_training_epochs` and `default_batch_size` in the `model_spec` object.

In [ ]:
model = question_answer.create(train_data, model_spec=spec)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Epoch 1/2
1067/1067 [==============================] - 2278s 2s/step - loss: 1.7592 - start_positions_loss: 1.6721 - end_positions_loss: 1.8463
Epoch 2/2
1067/1067 [==============================] - 2216s 2s/step - loss: 0.9264 - start_positions_loss: 0.9263 - end_positions_loss: 0.9265


Have a look at the detailed model structure.

In [ ]:
model.summary()

Model: "bert_span_labeler"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 384)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 384)]        0                                            
__________________________________________________________________________________________________
input_type_ids (InputLayer)     [(None, 384)]        0                                            
__________________________________________________________________________________________________
core_model (Functional)         [(None, 384, 512), ( 24581888    input_word_ids[0][0]             
                                                                 input_mask[0][0] 

## Evaluate the Customized Model

Evaluate the model on the validation data and get a dict of metrics including `f1` score and `exact match` etc. Note that metrics are different for SQuAD1.1 and SQuAD2.0.

In [ ]:
model.evaluate(validation_data)

INFO:tensorflow:Made predictions for 200 records.


INFO:tensorflow:Made predictions for 200 records.


INFO:tensorflow:Made predictions for 400 records.


INFO:tensorflow:Made predictions for 400 records.


INFO:tensorflow:Made predictions for 600 records.


...

INFO:tensorflow:Made predictions for 1000 records.


INFO:tensorflow:Made predictions for 1000 records.


INFO:tensorflow:Made predictions for 1200 records.


INFO:tensorflow:Made predictions for 1200 records.


{'exact_match': 0.5510204081632653, 'final_f1': 0.6304186176635158}

## Export to TensorFlow Lite Model

Convert the trained model to TensorFlow Lite model format with [metadata](https://www.tensorflow.org/lite/convert/metadata) so that you can later use in an on-device ML application. The vocab file are embedded in metadata. The default TFLite filename is `model.tflite`.

In many on-device ML application, the model size is an important factor. Therefore, it is recommended that you apply quantize the model to make it smaller and potentially run faster.
The default post-training quantization technique is dynamic range quantization for the BERT and MobileBERT models.

In [ ]:
model.export(export_dir='.')

INFO:tensorflow:Assets written to: /tmp/tmpnt3greeu/saved_model/assets


INFO:tensorflow:Assets written to: /tmp/tmpnt3greeu/saved_model/assets


INFO:tensorflow:Vocab file is inside the TFLite model with metadata.


INFO:tensorflow:Vocab file is inside the TFLite model with metadata.


INFO:tensorflow:Saved vocabulary in /tmp/tmpjk_ffz5n/vocab.txt.


...

INFO:tensorflow:The associated file that has been been packed to the model is:


INFO:tensorflow:['vocab.txt']


INFO:tensorflow:['vocab.txt']


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


You can use the TensorFlow Lite model file in the [bert_qa](https://github.com/tensorflow/examples/tree/master/lite/examples/bert_qa/android) reference app using [BertQuestionAnswerer API](https://www.tensorflow.org/lite/inference_with_metadata/task_library/bert_question_answerer) in [TensorFlow Lite Task Library](https://www.tensorflow.org/lite/inference_with_metadata/task_library/overview) by downloading it from the left sidebar on Colab.

The allowed export formats can be one or a list of the following:

*   `ExportFormat.TFLITE`
*   `ExportFormat.VOCAB`
*   `ExportFormat.SAVED_MODEL`

By default, it just exports TensorFlow Lite model with metadata. You can also selectively export different files. For instance, exporting only the vocab file as follows:

In [ ]:
model.export(export_dir='.', export_format=ExportFormat.VOCAB)

You can also evaluate the tflite model with the `evaluate_tflite` method. This step is expected to take a long time.

In [ ]:
model.evaluate_tflite('model.tflite', validation_data)

## Advanced Usage

The `create` function is the critical part of this library in which the `model_spec` parameter defines the model specification. The `BertQASpec` class is currently supported. There are 2 models: MobileBERT model, BERT-Base model. The `create` function comprises the following steps:

1. Creates the model for question answer according to `model_spec`.
2. Train the question answer model.

This section describes several advanced topics, including adjusting the model, tuning the training hyperparameters etc.

### Adjust the model

You can adjust the model infrastructure like parameters `seq_len` and `query_len` in the `BertQASpec` class.

Adjustable parameters for model:

* `seq_len`: Length of the passage to feed into the model.
* `query_len`: Length of the question to feed into the model.
* `doc_stride`: The stride when doing a sliding window approach to take chunks of the documents.
* `initializer_range`: The stdev of the truncated_normal_initializer for initializing all weight matrices.
* `trainable`: Boolean, whether pre-trained layer is trainable.

Adjustable parameters for training pipeline:

* `model_dir`: The location of the model checkpoint files. If not set, temporary directory will be used.
* `dropout_rate`: The rate for dropout.
* `learning_rate`: The initial learning rate for Adam.
* `predict_batch_size`: Batch size for prediction.
* `tpu`: TPU address to connect to. Only used if using tpu.


For example, you can train the model with a longer sequence length. If you change the model, you must first construct a new `model_spec`.

In [ ]:
new_spec = model_spec.get('mobilebert_qa')
new_spec.seq_len = 512

The remaining steps are the same. Note that you must rerun both the `dataloader` and `create` parts as different model specs may have different preprocessing steps.


### Tune training hyperparameters
You can also tune the training hyperparameters like `epochs` and `batch_size` to impact the model performance. For instance,

*   `epochs`: more epochs could achieve better performance, but may lead to overfitting.
*   `batch_size`: number of samples to use in one training step.

For example, you can train with more epochs and with a bigger batch size like:

```python
model = question_answer.create(train_data, model_spec=spec, epochs=5, batch_size=64)
```

### Change the Model Architecture

You can change the base model your data trains on by changing the `model_spec`. For example, to change to the BERT-Base model, run:

```python
spec = model_spec.get('bert_qa')
```

The remaining steps are the same.

### Customize Post-training quantization on the TensorFlow Lite model

[Post-training quantization](https://www.tensorflow.org/lite/performance/post_training_quantization) is a conversion technique that can reduce model size and inference latency, while also improving CPU and hardware accelerator inference speed, with a little degradation in model accuracy. Thus, it's widely used to optimize the model.

Model Maker library applies a default post-training quantization techique when exporting the model. If you want to customize post-training quantization, Model Maker supports multiple post-training quantization options using [QuantizationConfig](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/config/QuantizationConfig) as well. Let's take float16 quantization as an instance. First, define the quantization config.

```python
config = QuantizationConfig.for_float16()
```


Then we export the TensorFlow Lite model with such configuration.

```python
model.export(export_dir='.', tflite_filename='model_fp16.tflite', quantization_config=config)
```

# Read more

You can read our [BERT Question and Answer](https://www.tensorflow.org/lite/examples/bert_qa/overview) example to learn technical details. For more information, please refer to:

*   TensorFlow Lite Model Maker [guide](https://www.tensorflow.org/lite/guide/model_maker) and [API reference](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker).
*   Task Library: [BertQuestionAnswerer](https://www.tensorflow.org/lite/inference_with_metadata/task_library/bert_question_answerer) for deployment.
*   The end-to-end reference apps: [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/bert_qa/android) and [iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/bert_qa/ios).